In [ ]:
import numpy as np
import pandas as pd

In [2]:
root = '/Users/mdiaspinto/Documents/School/Python Data Science/Final Project/kaggle-churn'
df_raw = pd.read_parquet(root + '/data/train.parquet')
unused = ['status', 'firstName', 'lastName', 'ts', 'method', 'auth', 'userAgent']
df_raw.drop(columns=unused, inplace=True)
df_raw.head()

,gender,level,userId,page,sessionId,location,itemInSession,length,song,artist,time,registration
0,M,paid,1749042,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",278,524.32934,Ich mache einen Spiegel - Dream Part 4,Popol Vuh,2018-10-01 00:00:01,2018-08-08 13:22:21
992,M,paid,1749042,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",279,178.02404,Monster (Album Version),Skillet,2018-10-01 00:08:45,2018-08-08 13:22:21
1360,M,paid,1749042,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",280,232.61995,Seven Nation Army,The White Stripes,2018-10-01 00:11:43,2018-08-08 13:22:21
1825,M,paid,1749042,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",281,265.50812,Under The Bridge (Album Version),Red Hot Chili Peppers,2018-10-01 00:15:35,2018-08-08 13:22:21
2366,M,paid,1749042,NextSong,22683,"Dallas-Fort Worth-Arlington, TX",282,471.69261,Circlesong 6,Bobby McFerrin,2018-10-01 00:20:00,2018-08-08 13:22:21


In [3]:
root = '/Users/mdiaspinto/Documents/School/Python Data Science/Final Project/kaggle-churn'
df_test = pd.read_parquet(root + '/data/test.parquet')
df_test.drop(columns=unused, inplace=True)
df_test.head()

,gender,level,userId,page,sessionId,location,itemInSession,length,song,artist,time,registration
7,M,free,1465194,NextSong,22483,"New York-Newark-Jersey City, NY-NJ-PA",29,250.82730,Mockingbird,Eminem,2018-10-01 00:00:06,2018-09-27 17:29:36
54,M,free,1465194,Roll Advert,22483,"New York-Newark-Jersey City, NY-NJ-PA",30,NaN,None,None,2018-10-01 00:00:28,2018-09-27 17:29:36
477,M,free,1465194,NextSong,22483,"New York-Newark-Jersey City, NY-NJ-PA",31,355.78730,Thank You (Precious Memories Album Version),Ray Boltz,2018-10-01 00:04:16,2018-09-27 17:29:36
1170,M,free,1465194,NextSong,22483,"New York-Newark-Jersey City, NY-NJ-PA",32,191.68608,Mathletics,Foals,2018-10-01 00:10:11,2018-09-27 17:29:36
1552,M,free,1465194,NextSong,22483,"New York-Newark-Jersey City, NY-NJ-PA",33,275.25179,Proceed,The Roots,2018-10-01 00:13:22,2018-09-27 17:29:36


In [ ]:
def feature_builder(df: pd.DataFrame, cutoff_date: pd.Timestamp) -> pd.DataFrame:
    
    # Create a slice of the dataframe up to the cutoff date and makes userId the index
    df_slice = df[df['time'] < cutoff_date].copy()
    idx = pd.Index(np.sort(df['userId'].unique()), name='userId')
    final_df = pd.DataFrame(index=idx)

    # Get key features from users at cutoff date
    user_group = df.groupby('userId')
    final_df['level'] = user_group["level"].last().reindex(idx)
    final_df["days_registered"] = \
        (cutoff_date.normalize() - user_group["registration"].min().reindex(idx).dt.normalize()).dt.days.astype(int)

    # Group sessions and defines start and end for each one
    session_group = df_slice.groupby(['userId', 'sessionId']).agg(
        session_start=('time', 'min'),
        session_end=('time', 'max'),
        soung_count=("song", "count")
    )

    # Calculate session length in seconds
    session_group["session_length"] = (
    session_group["session_end"] - session_group["session_start"]
    ).dt.total_seconds()
    
    # Aggregate session statistics per user
    session_stats = session_group.groupby('userId').agg(
        num_sessions=('session_start', 'count'),
        avg_songs_per_session=('soung_count', 'mean'),
        avg_session_length=('session_length', 'mean'),
        days_since_last_session=('session_end', lambda x: (cutoff_date - x.max()).days),
    )

    # Convert to hours
    session_stats['avg_session_length'] /= 3600

    # Calculate proportion of activity on weekends
    df_slice['day'] = df_slice['time'].dt.dayofweek
    df_slice['weekend'] = df_slice['day'].isin([5, 6]).astype(int)
    final_df['weekend_perc'] = (df_slice.groupby('userId')['weekend'].sum()\
        /df_slice.groupby('userId')['weekend'].count()).reindex(idx, fill_value=0)
    final_df['weekend_perc'] *= 100

    # Calculate proportion of weekend days in the target window
    target_window = pd.date_range(start=cutoff_date + pd.Timedelta(days=1), periods=10)
    weekend_window_perc = (target_window.dayofweek.isin([5, 6])).sum()
    final_df['weekend_target_perc'] = weekend_window_perc * 10

    # Produce final dataframe for output
    final_df = final_df.join(session_stats.reindex(idx))
    num_features = ['num_sessions', 'avg_songs_per_session', \
                    'avg_session_length', 'days_since_last_session']
    final_df[num_features] = final_df[num_features].fillna(0)

    return final_df

In [14]:
test = feature_builder(df_raw, pd.Timestamp("2018-10-20"))
test.head()

,level,days_registered,weekend_perc,weekend_target_perc,num_sessions,avg_songs_per_session,avg_session_length,days_since_last_session
userId,,,,,,,,
1000025,paid,102,6.882793,30,17.0,97.764706,6.746552,1.0
1000035,paid,38,0.250627,30,7.0,46.285714,3.059206,0.0
1000083,paid,43,0.000000,30,11.0,45.545455,3.101742,7.0
1000103,paid,28,0.000000,30,1.0,39.000000,2.706667,15.0
1000164,paid,69,3.649635,30,9.0,37.111111,2.532253,0.0


In [6]:
def label_builder(df: pd.DataFrame,
                 cutoff_date: pd.Timestamp,
                 window_size: int = 10,
                 buffer: int = 3) -> pd.Series:
    
    # Define the time window including buffer period
    window_end = cutoff_date + pd.Timedelta(days=window_size)
    buffer_end = window_end + pd.Timedelta(days=buffer)
    window_users = df.loc[df["time"] <= cutoff_date, "userId"].unique()

    # Get the cancellation time for each user
    cancel_time = (
        df.loc[df["page"] == "Cancellation Confirmation"]
          .groupby("userId")["time"]
          .min()
          .reindex(window_users)
    )

    # Set target labels based on cancellation time
    y = pd.Series(0, index=window_users, name="target")
    y[cancel_time <= cutoff_date] = np.nan
    y[(cancel_time > window_end) & (cancel_time <= buffer_end)] = np.nan
    y[(cancel_time > cutoff_date) & (cancel_time <= window_end)] = 1

    return y

In [7]:
test_y = label_builder(df_raw, pd.Timestamp("2018-10-20"))
test_y.head()

1749042    1.0
1563081    0.0
1697168    0.0
1222580    NaN
1714398    0.0
Name: target, dtype: float64

In [ ]:
def window_builder(df: pd.DataFrame,
                start_date,
                end_date,
                *,
                step_days: int = 7,
                horizon_days: int = 10,
                buffer_days: int = 3,
                corr_threshold: float = 0.95,
                categorical_cols=("level",),
                verbose: bool = True):

    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)

    all_windows = []
    current = start_date

    if verbose:
        print(f"Generating rolling training data from {start_date.date()} to {end_date.date()}...")

    while current <= end_date:
        if verbose:
            print(f"  - Processing window: {current.date()}")

        feats = feature_builder(df, current)
        labels = label_builder(df, current, window_size=horizon_days, buffer=buffer_days)

        labels = labels.reindex(feats.index)
        mask = labels.notna()

        window = feats.loc[mask].copy()
        window["target"] = labels.loc[mask].astype(int)
        window["snapshot_date"] = current

        all_windows.append(window)
        current += pd.Timedelta(days=step_days)

    rolling_df = pd.concat(all_windows, axis=0)

    # Drop userId index
    groups = rolling_df.index.to_numpy()
    rolling_df = rolling_df.reset_index(drop=True)

    # Define X and y
    X = rolling_df.drop(columns=["target", "snapshot_date"], errors="ignore")
    y = rolling_df["target"].astype(int)

    # Mark categoricals as category dtype
    for c in categorical_cols:
        if c in X.columns:
            X[c] = X[c].astype("category")

    # Drop highly correlated numeric columns
    dropped_cols = []
    if corr_threshold is not None:
        X_num = X.select_dtypes(include=[np.number])
        if X_num.shape[1] >= 2:
            corr = X_num.corr().abs()
            upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))
            dropped_cols = [col for col in upper.columns if (upper[col] > corr_threshold).any()]
            if verbose:
                print(f"Dropping correlated (>{corr_threshold}): {dropped_cols}")
            X = X.drop(columns=dropped_cols, errors="ignore")

    return X, y, groups, rolling_df, dropped_cols


start_dt = pd.Timestamp("2018-10-01")
end_dt = pd.Timestamp("2018-11-05")

X_rolling, y_rolling, groups_rolling, rolling_df, to_drop = window_builder(
    df_raw,
    start_dt,
    end_dt,
    step_days=7,
    horizon_days=10,
    buffer_days=3,
    corr_threshold=0.95,
    categorical_cols=("level",),
    reset_index=True,
    verbose=True,
 )

print(f"Total Samples: {len(rolling_df)}")

Generating rolling training data from 2018-10-01 to 2018-11-05...
  - Processing window: 2018-10-01


/var/folders/69/x1vfxvpd1_q07my0mb9rc8tm0000gn/T/ipykernel_6893/793607365.py:52: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  final_df[num_features] = final_df[num_features].fillna(0)


  - Processing window: 2018-10-08
  - Processing window: 2018-10-15
  - Processing window: 2018-10-22
  - Processing window: 2018-10-29
  - Processing window: 2018-11-05
Dropping correlated (>0.95): ['avg_session_length']
Total Samples: 72948
